In [1]:
import torch
import torch.nn as nn
from torchvision import models, transforms
from PIL import Image

# Define model structure

In [2]:
# Model Definition
class KeyPointModel(nn.Module):
    def __init__(self):
        super(KeyPointModel, self).__init__()
        self.resnet = models.resnet50(pretrained=True)
        self.resnet.fc = nn.Linear(self.resnet.fc.in_features, 24)  # 12 key points (24 coordinates) - add fuly connected layer

    def forward(self, x):
        return self.resnet(x)


# load data

In [3]:
import os
import pandas as pd
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision import models
from PIL import Image
from torch.utils.data import Dataset, DataLoader

# Instantiate the model
model = KeyPointModel()
model.train()  # Set to training mode

# Dataset Definition
class KeyPointDataset(Dataset):
    def __init__(self, image_paths, key_points, transform=None):
        self.image_paths = image_paths
        self.key_points = key_points
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image = Image.open(self.image_paths[idx])
        key_point = self.key_points[idx]

        if self.transform:
            image = self.transform(image)

        return image, key_point

# Function to load dataset from a specified folder
def load_dataset(folder_path):
    # Load key points from the CSV file
    csv_file = os.path.join(folder_path, 'augmented_labels.csv')  # Change to your CSV file name
    keypoint_data = pd.read_csv(csv_file)

    # Create lists for images and key points
    image_paths = []
    key_points = []

    # Iterate through the CSV to get paths and key points
    for index, row in keypoint_data.iterrows():
        image_name = row['image_name']  # Replace with the actual column name in your CSV
        keypoint = row[1:].values.astype(float)  # Assuming the first column is the image name
        image_path = os.path.join(folder_path, f"{image_name}")  # Assuming images are in PNG format

        image_paths.append(image_path)
        key_points.append(torch.tensor(keypoint))

    return image_paths, key_points

# Base path for your project
base_path = os.path.expanduser('~/Documents/GitHub/ViT_facemap/ViT-pytorch/projects/Facemap/data')

# Paths to your train and test folders
train_folder = os.path.join(base_path, 'train', 'augmented_data')
test_folder = os.path.join(base_path, 'test', 'augmented_data')

# Load datasets
train_image_paths, train_key_points = load_dataset(train_folder)
test_image_paths, test_key_points = load_dataset(test_folder)

# Transformations
transform = transforms.Compose([
    #transforms.Resize(256),
    #transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Create DataLoaders
train_dataset = KeyPointDataset(train_image_paths, train_key_points, transform)
train_dataloader = DataLoader(train_dataset, batch_size=20, shuffle=True)

test_dataset = KeyPointDataset(test_image_paths, test_key_points, transform)
test_dataloader = DataLoader(test_dataset, batch_size=20, shuffle=False)

# Define the loss function and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


/Users/annastuckert/anaconda3/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/annastuckert/anaconda3/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


# Train network

In [4]:
# This code should include both validation and training loss 
import torch
import wandb
import os

# Initialize WandB
wandb.init(project="Facemap_ResNet", name="50_epochs")

# Define your hyperparameters and log them to WandB
config = wandb.config
config.num_epochs = 50
config.learning_rate = 0.001
config.batch_size = 20

# Check if CUDA is available and set device accordingly
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Move model to the appropriate device
model = model.to(device)

# Initialize optimizer, scheduler, and loss function
optimizer = torch.optim.AdamW(model.parameters(), lr=config.learning_rate)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
criterion = torch.nn.MSELoss()

# Training Loop
num_epochs = config.num_epochs  # Number of epochs
global_step = 0  # To keep track of total steps across epochs

for epoch in range(num_epochs):
    running_loss = 0.0
    model.train()  # Set the model to training mode
    
    # Training loop
    for batch_idx, (images, targets) in enumerate(train_dataloader):
        # Move data to the GPU
        images = images.to(device)
        targets = targets.float().to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        global_step += 1  # Increment global step

        # Log the loss for the batch to WandB
        wandb.log({"Batch Loss": loss.item(), "global_step": global_step})

        # Print progress every 10 batches (adjust as needed)
        if batch_idx % 10 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Batch [{batch_idx}/{len(train_dataloader)}], Loss: {loss.item():.4f}')

    # Calculate average training loss for the epoch
    avg_train_loss = running_loss / len(train_dataloader)
    print(f'Epoch [{epoch+1}/{num_epochs}], Average Training Loss: {avg_train_loss:.4f}')
    
    # Validation loop after each epoch
    model.eval()  # Set the model to evaluation mode
    val_running_loss = 0.0
    with torch.no_grad():
        for batch_idx, (images, targets) in enumerate(test_dataloader):
            # Move data to the GPU
            images = images.to(device)
            targets = targets.float().to(device)

            outputs = model(images)
            val_loss = criterion(outputs, targets)
            val_running_loss += val_loss.item()

    # Calculate average validation loss for the epoch
    avg_val_loss = val_running_loss / len(test_dataloader)
    print(f'Epoch [{epoch+1}/{num_epochs}], Average Validation Loss: {avg_val_loss:.4f}')
    
    # Log epoch-level metrics (training and validation loss) to WandB
    wandb.log({
        "Epoch": epoch + 1,
        "Average Training Loss": avg_train_loss,
        "Average Validation Loss": avg_val_loss
    })
    
    # Step the learning rate scheduler
    scheduler.step()

# Save the trained model
model_path = 'R50_model.pth'
torch.save(model.state_dict(), model_path)

# Save the model to WandB
wandb.save(model_path)


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: anna-stuckert (anna-stuckert-university-of-st-andrews). Use `wandb login --relogin` to force relogin


Epoch [1/50], Batch [0/45], Loss: 10793.7461
Epoch [1/50], Batch [10/45], Loss: 7963.4580
Epoch [1/50], Batch [20/45], Loss: 5120.9150


KeyboardInterrupt: 

# Plotting predictions via forward pass

In [ ]:
import torch
from torchvision import transforms
from PIL import Image
import matplotlib.pyplot as plt

# Load the trained model
model = KeyPointModel()
model.load_state_dict(torch.load('R50_model.pth'))
model.eval()  # Set to evaluation mode

# Function to preprocess an image
def preprocess_image(image_path):
    image = Image.open(image_path)
    transform = transforms.Compose([
        #transforms.Resize(256),
        #transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    image = transform(image).unsqueeze(0)  # Add batch dimension
    return image

# Function to visualize predictions
def visualize_predictions(image_path, keypoints):
    # Load and display the original image
    original_image = Image.open(image_path)
    plt.imshow(original_image)
    
    # Convert keypoints to a numpy array and plot them
    x_coords = keypoints[0::2]
    y_coords = keypoints[1::2]
    plt.scatter(x_coords, y_coords, c='red', marker='x')
    plt.title('Predicted Key Points')
    plt.axis('off')
    plt.show()

# Path to the image you want to predict key points for
# Base path for your project
base_path = os.path.expanduser('~/Documents/GitHub/ViT_facemap/ViT-pytorch/projects/Facemap/data')

# Directory containing the images
image_folder = os.path.join(base_path, 'test', 'augmented_data')

# Get the first image in the folder
image_files = [f for f in os.listdir(image_folder) if f.endswith('.jpg') or f.endswith('.png')]
if not image_files:
    raise ValueError("No images found in the specified folder.")
image_path = os.path.join(image_folder, image_files[0])


# Preprocess the image
input_image = preprocess_image(image_path)

# Perform a forward pass to get predictions
with torch.no_grad():
    predicted_keypoints = model(input_image)

# Convert predictions to numpy array and detach from the graph
predicted_keypoints = predicted_keypoints.squeeze().numpy()

# Visualize the predicted key points
visualize_predictions(image_path, predicted_keypoints)


# From hereon I was trying to make the SHAP explainer work - for the time being I have kept the code here for reference, but the 'working' kernel explainer is in the script kernelSHAP.ipynb

## SHAP maybe works?

In [ ]:
#!pip install shap
import os
import shap
import numpy as np
import torch
from torchvision import transforms
from PIL import Image

# Load the trained model
model = KeyPointModel()
model.load_state_dict(torch.load('keypoint_model.pth'))
model.eval()

# Prepare the transform
transform = transforms.Compose([
    #transforms.Resize(256),
    #transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Function to prepare input for SHAP
def preprocess_image(image_path):
    image = Image.open(image_path)
    image = transform(image).unsqueeze(0)  # Add batch dimension
    return image

# Directory containing the images
# Base path for your project
base_path = os.path.expanduser('~/Documents/GitHub/ViT_facemap/ViT-pytorch/projects/Facemap/data')

# Directory containing the images
image_folder = os.path.join(base_path, 'test', 'augmented_data')

# Get the first image in the folder
image_files = [f for f in os.listdir(image_folder) if f.endswith('.jpg') or f.endswith('.png')]
if not image_files:
    raise ValueError("No images found in the specified folder.")
first_image_path = os.path.join(image_folder, image_files[0])

# Prepare the image for SHAP analysis
images = preprocess_image(first_image_path)

# Reshape images to 2D (num_samples, num_features)
images_reshaped = images.view(images.size(0), -1)  # Flatten each image

# Create a SHAP explainer
def model_predict(input_data):
    input_data = torch.tensor(input_data, dtype=torch.float32)  # Ensure input is a tensor
    with torch.no_grad():
        return model(input_data.view(-1, 3, 224, 224)).numpy()  # Reshape back to original image size

# Initialize the Kernel Explainer with flattened images
explainer = shap.KernelExplainer(model_predict, images_reshaped.numpy())

# Calculate SHAP values
shap_values = explainer.shap_values(images_reshaped.numpy())

# Visualize SHAP values for the first image
shap.initjs()

# Remove the batch dimension for visualization
images_for_plotting = images.squeeze(0).permute(1, 2, 0).numpy()  # Shape to (224, 224, 3)

shap.image_plot(shap_values, images_for_plotting)


# SHAP based on each KP

https://shap.readthedocs.io/en/latest/example_notebooks/image_examples/image_classification/Explain%20ResNet50%20using%20the%20Partition%20explainer.html 

In [ ]:
import os
import pandas as pd
import torch
from torchvision import transforms
from PIL import Image

# Load the trained model
model = KeyPointModel()
model.load_state_dict(torch.load('keypoint_model.pth'))
model.eval()

# Function to load dataset from a specified folder
def load_dataset(folder_path):
    # Load key points from the CSV file
    csv_file = os.path.join(folder_path, 'augmented_labels.csv')  # Change to your CSV file name
    keypoint_data = pd.read_csv(csv_file)

    # Create lists for images and key points
    image_paths = []
    key_points = []

    # Iterate through the CSV to get paths and key points
    for index, row in keypoint_data.iterrows():
        image_name = row['image_name']  # Replace with the actual column name in your CSV
        keypoint = row[1:].values.astype(float)  # Assuming the first column is the image name
        image_path = os.path.join(folder_path, f"{image_name}")  # Assuming images are in PNG format

        image_paths.append(image_path)
        key_points.append(torch.tensor(keypoint))

    return image_paths, key_points

# Function to load and preprocess images
def load_and_preprocess_images(image_paths):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),  # Resize to model's input size
        transforms.ToTensor(),  # Convert to tensor
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize for ResNet
    ])
    
    images = []
    for path in image_paths:
        image = Image.open(path).convert('RGB')  # Load image and convert to RGB
        image = transform(image)  # Apply transformations
        images.append(image)
    
    return torch.stack(images)  # Stack images into a single tensor

# Base path for your project
base_path = os.path.expanduser('~/Documents/GitHub/ViT_facemap/ViT-pytorch/projects/Facemap/data')

# Paths to your train and test folders
train_folder = os.path.join(base_path, 'train', 'augmented_data')
test_folder = os.path.join(base_path, 'test', 'augmented_data')

# Load datasets
train_image_paths, train_key_points = load_dataset(train_folder)
test_image_paths, test_key_points = load_dataset(test_folder)

# Load and preprocess images
X_train = load_and_preprocess_images(train_image_paths)
X_test = load_and_preprocess_images(test_image_paths)

# Convert key points to tensors
y_train = torch.stack(train_key_points)
y_test = torch.stack(test_key_points)


In [ ]:
import shap
import numpy as np
import torch

# Check if CUDA is available and set device accordingly
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Move model to the appropriate device
model = model.to(device)

# Assuming 'X_train' is your training data
# Ensure that X_train is a PyTorch tensor and move it to the GPU
X_train_tensor = torch.tensor(X_train).to(device)

# Use a very small sample of data to avoid OOM error
X_train_sample = X_train_tensor[:10]  # Adjust based on your needs

# Clear GPU memory
torch.cuda.empty_cache()

# Create the SHAP explainer
explainer = shap.GradientExplainer(model.cpu(), X_train_sample.cpu())  # Use CPU for SHAP

# Use torch.no_grad() to save memory during SHAP value computation
with torch.no_grad():
    shap_values = explainer.shap_values(X_train_sample.cpu())

# Convert SHAP values back to numpy if needed
shap_values_np = [val.detach().numpy() for val in shap_values]

# Now you can use shap_values_np for further analysis or visualization





In [ ]:
# 'X_test' is your test data for which you want explanations
shap_values = explainer.shap_values(X_test)


In [ ]:
# Compute SHAP values in smaller batches
shap_values = []
batch_size = 1  # Adjust this value based on your available memory

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
X_test = X_test.to(device)


with torch.no_grad():
    for i in range(0, X_test.shape[0], batch_size):
        batch_shap_values = explainer.shap_values(X_test[i:i + batch_size])
        shap_values.extend(batch_shap_values)
        torch.cuda.empty_cache()  # Clear memory after each batch if necessary



In [ ]:
for i in range(12):  # Loop through each key point
    shap.initjs()
    shap.summary_plot(shap_values[i], X_test, feature_names=your_feature_names)


# Start over

In [ ]:
import torch
import shap

# Load the trained model
model = KeyPointModel()
model.load_state_dict(torch.load('R50_model_shap2.pth', map_location=torch.device('cpu')))
model.eval()
preprocessed_image = X_test[0]
# Load a sample image or batch of images (ensure the preprocessing is the same as in training)
sample_image = torch.tensor(preprocessed_image).unsqueeze(0)  # add batch dimension if needed

print(sample_image.shape)


In [ ]:
class ResNet50Wrapper(torch.nn.Module):
    def __init__(self, model, keypoint_idx):
        super().__init__()
        self.model = model
        self.keypoint_idx = keypoint_idx

    def forward(self, x):
        output = self.model(x)
        # Extracting specific keypoint (e.g., keypoint 1 corresponds to output 0 and 1 for x and y)
        return output[:, self.keypoint_idx*2:self.keypoint_idx*2+2]

# Wrap the model to focus on a specific key point, e.g., keypoint 0 (x1, y1)
wrapped_model = ResNet50Wrapper(model, keypoint_idx=0)  # Adjust keypoint_idx for different keypoints


In [ ]:
# SHAP Gradient Explainer requires the model and the background dataset (a set of representative images)
#background = torch.randn(50, 3, 224, 224)  # Random background data (replace with actual representative images if available)
background = X_train[:100]
# Create a SHAP Gradient explainer
explainer = shap.GradientExplainer(wrapped_model, background)

# Compute SHAP values for the keypoint of interest (e.g., keypoint 0)
shap_values = explainer.shap_values(sample_image)


In [ ]:
device = torch.device('cuda:0')
background = X_train[:100].to(device)
test_image = X_test[0].to(device)

e = shap.DeepExplainer(model, background)
shap_values = e.shap_values(test_image)


In [ ]:
# Assuming you want to visualize the first keypoint
keypoint_index = 0  # Change this index to visualize a different keypoint

# Extract the SHAP values for the specific keypoint
shap_values_keypoint = shap_values[keypoint_index]  # This will have shape (224, 224, 2)

# Since shap_values_keypoint is already in (H, W, 2), we can directly get the first channel (x or y) for visualization
shap_values_x = shap_values_keypoint[..., 0]  # Shape (224, 224)
shap_values_y = shap_values_keypoint[..., 1]  # Shape (224, 224)

# Convert the sample image to a NumPy array with the correct shape (H, W, C)
sample_image_numpy = sample_image.squeeze(0).detach().cpu().numpy().transpose(1, 2, 0)

# Ensure the values are in the valid range for images (e.g., 0-255)
sample_image_numpy = (sample_image_numpy * 255).astype(np.uint8)  # Assuming normalization was done earlier

# Visualize the SHAP values for the selected keypoint's x coordinate as a heatmap overlay
shap.image_plot([shap_values_x], sample_image_numpy)

# If you want to visualize the y coordinate as well
shap.image_plot([shap_values_y], sample_image_numpy)


In [ ]:
# start over

from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
import json
import shap
import tensorflow as tf

# load pre-trained model and choose two images to explain
model = ResNet50(weights='imagenet')
def f(X):
    tmp = X.copy()
    preprocess_input(tmp)
    return model(tmp)
X, y = shap.datasets.imagenet50()

# load the ImageNet class names as a vectorized mapping function from ids to names
url = "https://s3.amazonaws.com/deep-learning-models/image-models/imagenet_class_index.json"
with open(shap.datasets.cache(url)) as file:
    class_names = [v[1] for v in json.load(file).values()]

# define a masker that is used to mask out partitions of the input image, this one uses a blurred background
masker = shap.maskers.Image("inpaint_telea", X[0].shape)

# By default the Partition explainer is used for all  partition explainer
explainer = shap.Explainer(f, masker, output_names=class_names)

# here we use 500 evaluations of the underlying model to estimate the SHAP values
shap_values = explainer(X[1:3], max_evals=500, batch_size=50, outputs=shap.Explanation.argsort.flip[:1])
shap.image_plot(shap_values)